In [1]:
import fitz
import re
import pandas as pd
from classes import *

In [2]:
path = r'C:\Users\SWV224\BOLLORE\XPF - Documents\MAINTENANCE\PYTHON\pdf-parser\manifests'

doc330 = path + r'\H330 X-PRESS MULHACEN 10.12.2021.pdf'
doc331 = path + r'\H331 X-PRESS MULHACEN 25.12.2021.pdf'
doc332 = path + r'\H332 X-PRESS MULHACEN 11.1.2022.pdf'
doc356 = path + r'\H356 X-PRESS MULHACEN 28.1.2022.pdf'
doc420 = path + r'\H420 X-PRESS MULHACEN 07.03.2022.pdf'
doc422 = path + r'\H422 X-PRESS MULHACEN 21.3.2022.pdf'


doc = fitz.open(doc422)

In [3]:
shipper = (33, 140)
unit = (222, 290)
goods = (312, 445)
netw = (468, 502)
tare = (510, 547)
pod = (460, 500)
base_triangle = (33, 100, 547, 580)

In [4]:
total_starts = list()  # lista över rektanglar för "Shipper :" på alla blad
total_stops = []    # lista över rektanglar för "Units :" på alla blad
total_height = 0.0  # loop över summan av höjderna på alla blad
total_words = []    # lista över alla dokumentets ord på alla blad
rect_pods = []     # lista över rektanglar för "Port of Discharge" på alla blad
words_in_rectangles = [] # lista för ord inom rektang*larna, utan "headers"
total_pods = [] # lista över rektanglar och namn för PODs
list_of_base_triangles = []
list_of_rectangles = list()
list_of_tod_rectangles = []
final_tod_list = []

for page in doc:

    # starts
    starts = page.search_for("Shipper :")
    for rect in starts:
        total_starts.append(rect + (0, total_height, 0, total_height)) # lägger till på höjden för y0 och y1


    # stops
    stops = page.search_for("Units :")
    for rect in stops:
        total_stops.append(rect + (0, total_height, 0, total_height))

    # total_words
    words_in_page = page.get_text("words")
    for words in words_in_page:
        total_words.append([words[0], words[1] + total_height, words[2], words[3] + total_height, words[4]])

    # rect_pods
    pods = page.search_for('Port of discharge')
    for rect in pods:      
        rect_pods.append(rect + (0, total_height + 12, 0, total_height + 16)) # lägger till på y0 och y1 för att hitta POD: y0=12.20998, y1=16.08598
    
    # words_in_rectangles
    base_triangle = (33, 100 + total_height, 547, 580 + total_height) # Utgår från basrektangel (33, 100, 547, 580) och lägger på totalhöjden
    words_in_rectangles += [word for word in total_words if fitz.Rect(word[:4]).intersects(base_triangle)] # Tar fram alla ord inom rektangeln ovan
    list_of_base_triangles.append(base_triangle)
    
    total_height += page.rect.height # lägger till totalhöjden varje loop

pod_names = [' '.join([words[4] for words in total_words if fitz.Rect(words[:4]).intersects(pod)]) for pod in rect_pods] # Tar fram alla POD is en lista

rectangle = lambda x: fitz.Rect(33, total_starts[x][1], 580, total_stops[x][3])

for num, starts in enumerate(total_starts):
    list_of_rectangles.append(rectangle(num))

for rectangles, tod in zip(list_of_base_triangles, pod_names):
    list_of_tod_rectangles.append([rectangles[0], rectangles[1], rectangles[2], rectangles[3], tod])

for rect in list_of_rectangles:
    counter = 0
    for tod in list_of_tod_rectangles:
        if tod[3] // rect[3] > 0 and counter == 0:
            final_tod_list.append(tod[4])
            counter == 1
            break

In [ ]:
def sub_loop():

    rectangle = (shipper[0], start[1], shipper[1], stop[3])
    same_row = start[1]
    list_of_data = []
    str_of_words = ""
    counter = 0

    for word in words_in_rectangles:
        if fitz.Rect(word[:4]).intersects(rectangle):
            
            if same_row == word[1]:                                
                str_of_words += " " + word[4]

            elif same_row != word[1]:                             
                list_of_data.append(str_of_words.strip())
                str_of_words = word[4]

            same_row = word[1]

    for str in list_of_data:
        #matching SHIPPER
        shipper_match = re.search('Shipper :', str)

        if counter == 1:
            counter = 0
            return str

        elif shipper_match:
            counter = 1


    pass

In [ ]:

start = total_starts[0]
stop = total_stops[0]
shipper = Coordinates(x0=33, x1=140).update(start, stop)
list_of_data = []
str_of_words = ""
counter = 0

for num, word in enumerate(words_in_rectangles):
    if fitz.Rect(word[:4]).intersects(shipper):
        
        if words_in_rectangles[num-1][1] == words_in_rectangles[num][1]:                                
            str_of_words += " " + word[4]

        elif words_in_rectangles[num-1][1] != words_in_rectangles[num][1]:                             
            list_of_data.append(str_of_words.strip())
            str_of_words = word[4]

for str in list_of_data:
    #matching SHIPPER
    shipper_match = re.search('Shipper :', str)

    if counter == 1:
        counter = 0
        print(str)

    elif shipper_match:
        counter = 1